사전준비

In [9]:
import tensorflow as tf
import os
from PIL import Image
import numpy as np

In [384]:
path="C:/Users/82109/Desktop/datasets"
savePath=""

이미지 전처리

In [310]:
#해당 디렉토리의 모든 파일명을 가져옴

gifPath=[]
#경로 반환
def GetFilePath(path,end=".gif"):
  gifFileList=os.listdir(path)
  gifPath=[]
  for name in gifFileList:
    if name.endswith(tuple(end)):
      gifPath.append(os.path.join(path,name))
  return gifPath


#못쓰는 데이터를 걸러줌
def PreprocessGif(path,frame=5):
  gif=Image.open(path)
  size=gif.n_frames
  gif.close()
  if size<frame:
    print(path,": ",size,"사용불가능")
    os.remove(path=path)
  else:
    print(path,": ",size," 사용가능")

#gif를 읽고 넘파이 배열로 노멀라이즈해줌
def LoadGif(path, paddingSize=32):
  gif=Image.open(path)
  flip=np.random.randint(0,7,1)[0]

  images=[]
  for i in range(1,gif.n_frames):
    gif.seek(i)
    temp=gif.transpose(flip).convert("RGBA")
    temp=np.array(temp)
    height=(paddingSize-temp.shape[0]%paddingSize)%paddingSize
    width=(paddingSize-temp.shape[1]%paddingSize)%paddingSize
    temp=np.pad(temp, pad_width=((0,height),(0,width),(0,0)),mode="constant",constant_values=0)
    images.append(temp)
  gif.close()
  return np.array(images)/255.0

#인풋데이터와 아웃풋 데이터를 분리
def Divide(arr):
  evens=arr[0::2]
  odds=arr[1::2]
  if evens.shape[0] != odds.shape[0]:
    evens=evens[0:-1]
  return [evens,odds]


#사용예제 

#gifPath=GetFilePath(path) 
# for i in gifPath:
#   PreprocessGif(i)

#arr=LoadGif(gifPath[3])
#input, output=Divide(arr)
#print(arr.shape ,input.shape, output.shape)


In [408]:
#데이터셋 제너레이터 생성
def DatasetGenerater(gifPath):
  #gif파일을 반환
  for i in gifPath:
      x,y=Divide(LoadGif(i))  
      yield (x,y)

#사용불가능 파일 전처리
gifPath=GetFilePath(path) 
for i in gifPath:
  PreprocessGif(i)
gifPath=GetFilePath(path)


C:/Users/82109/Desktop/datasets\1.gif :  32  사용가능
C:/Users/82109/Desktop/datasets\2.gif :  31  사용가능
C:/Users/82109/Desktop/datasets\3.gif :  128  사용가능
C:/Users/82109/Desktop/datasets\smoke_loop_animation_by_alexredfish_d8uuq6h.gif :  45  사용가능
C:/Users/82109/Desktop/datasets\tomoe_kickslash_riser_animation_by_hybridmink_d9yq8fu.gif :  15  사용가능
C:/Users/82109/Desktop/datasets\train_by_kirokaze_d9q4jxs.gif :  29  사용가능
C:/Users/82109/Desktop/datasets\t_crouch_kicks_by_hybridmink_d952wjw.gif :  13  사용가능


In [409]:
trainDataset=tf.data.Dataset.from_generator(DatasetGenerater,
                               args=[gifPath], output_types=(tf.float32,tf.float32),
                               output_shapes = ((None, None,None,4),(None, None,None,4)))
#(inputImages, outputImages)
trainDataset=trainDataset.shuffle(10).batch(1).prefetch(1)


unet 모델 생성

In [389]:

def seperableConv(filter, input):
    depthwise=tf.keras.layers.Conv3D(input.shape[-1],3,padding="same",groups=input.shape[-1])(input)
    pointwise=tf.keras.layers.Conv3D(filter,1,padding="same")(depthwise)
    return pointwise
    
def block(filter,input):
    #conv1=tf.keras.layers.Conv3D(filter,3,padding="same")(input)
    conv1=seperableConv(filter,input)
    layerNorm1=tf.keras.layers.LayerNormalization()(conv1)
    swishAct1=tf.keras.layers.Activation("swish")(layerNorm1)

    #conv2=tf.keras.layers.Conv3D(filter,3,padding="same")(swishAct1)
    conv2=seperableConv(filter,swishAct1)
    layerNorm2=tf.keras.layers.LayerNormalization()(conv2)
    swishAct2=tf.keras.layers.Activation("swish")(layerNorm2)
    
    return swishAct2


def unetModel(inputShape=(None, None, None, 4)):
    inputImage=tf.keras.Input(shape=inputShape)
    inputRate=tf.keras.Input(shape=(inputShape[0],inputShape[1],inputShape[2],1))
    input=tf.keras.layers.Concatenate()([inputImage,inputRate])
    
    #인코딩
    e1=block(32,input)
    e1Pooling=tf.keras.layers.MaxPooling3D(pool_size=(1, 2, 2))(e1)
    
    e2=block(64,e1Pooling)
    e2Pooling=tf.keras.layers.MaxPooling3D(pool_size=(1, 2, 2))(e2)
    
    e3=block(128,e2Pooling)
    e3Pooling=tf.keras.layers.MaxPooling3D(pool_size=(1, 2, 2))(e3)
    
    e4=block(256,e3Pooling)
    e4Pooling=tf.keras.layers.MaxPooling3D(pool_size=(1, 2, 2))(e4)
    
    e5=block(512,e4Pooling)
    e5Pooling=tf.keras.layers.MaxPooling3D(pool_size=(1, 2, 2))(e5)
    
    #중간
    bottleNeck=block(1024,e5Pooling)

    d5UpSampling=tf.keras.layers.UpSampling3D(size=(1,2,2))(bottleNeck)
    d5Transpose=seperableConv(512,d5UpSampling)
    d5Concatenate=tf.keras.layers.Concatenate()([d5Transpose,e5])
    d5=block(512,d5Concatenate)
    
    d4UpSampling=tf.keras.layers.UpSampling3D(size=(1,2,2))(d5)
    d4Transpose=seperableConv(256,d4UpSampling)
    d4Concatenate=tf.keras.layers.Concatenate()([d4Transpose,e4])
    d4=block(256,d4Concatenate)
    
    d3UpSampling=tf.keras.layers.UpSampling3D(size=(1,2,2))(d4)
    d3Transpose=seperableConv(128,d3UpSampling)
    d3Concatenate=tf.keras.layers.Concatenate()([d3Transpose,e3])
    d3=block(128,d3Concatenate)
    
    d2UpSampling=tf.keras.layers.UpSampling3D(size=(1,2,2))(d3)
    d2Transpose=seperableConv(64,d2UpSampling)
    d2Concatenate=tf.keras.layers.Concatenate()([d2Transpose,e2])
    d2=block(64,d2Concatenate)
    
    d1UpSampling=tf.keras.layers.UpSampling3D(size=(1,2,2))(d2)
    d1Transpose=seperableConv(32,d1UpSampling)
    d1Concatenate=tf.keras.layers.Concatenate()([d1Transpose,e1])
    d1=block(32,d1Concatenate)
    
    outputImage=seperableConv(4,d1)
    
    return tf.keras.Model([inputImage,inputRate],outputImage)


디퓨전 예시

In [404]:
class DiffusionModel(tf.keras.Model):
    def __init__(self,network:tf.keras.Model,batchSize=1):
        super().__init__()
        self.network= network
        self.batchSize=batchSize
        
        
    def train_step(self, images):
        
        noises = tf.random.normal(
            shape=(self.batchSize, images.shape[1], images.shape[2], images.shape[3])
            )
        
        
        # # sample uniform random diffusion times
        # diffusion_times = tf.random.uniform(
        #     shape=(1, 1, 1, 1), minval=0.0, maxval=1.0
        # )
        # noise_rates, signal_rates = self.diffusion_schedule(diffusion_times)
        # # mix the images with noises accordingly
        # noisy_images = signal_rates * images + noise_rates * noises

        # with tf.GradientTape() as tape:
        #     # train the network to separate noisy images to their components
        #     pred_noises, pred_images = self.denoise(
        #         noisy_images, noise_rates, signal_rates, training=True
        #     )

        #     noise_loss = self.loss(noises, pred_noises)  # used for training
        #     image_loss = self.loss(images, pred_images)  # only used as metric

        # gradients = tape.gradient(noise_loss, self.network.trainable_weights)
        # self.optimizer.apply_gradients(zip(gradients, self.network.trainable_weights))

        # self.noise_loss_tracker.update_state(noise_loss)
        # self.image_loss_tracker.update_state(image_loss)

        # # track the exponential moving averages of weights
        # for weight, ema_weight in zip(self.network.weights, self.ema_network.weights):
        #     ema_weight.assign(ema * ema_weight + (1 - ema) * weight)

        # # KID is not measured during the training phase for computational efficiency
        # return {m.name: m.result() for m in self.metrics[:-1]}


In [406]:
network=unetModel(inputShape=(None,None,None,4))
model=DiffusionModel(network)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=savePath, 
    verbose=1, 
    save_weights_only=True,
    )
model.compile(
    tf.keras.optimizers.experimental.AdamW(
        learning_rate=1e-3, weight_decay=1e-4
    ),
    loss=tf.keras.losses.mean_squared_error,
              metrics=['accuracy'])

model.fit()

모델 저장 복구

In [402]:
savePath="C:/Users/82109/Desktop/새 폴더/sup"

model=unetModel(inputShape=(None,None,None,4))
model.save_weights(filepath=savePath)

model=unetModel(inputShape=(None,None,None,4))
model.load_weights(filepath=savePath)